In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import curve_fit
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
dose_resp = pd.read_csv("DOSERESP.csv").drop([
    "RELEASE_DATE", 
    "PREFIX",
    "PANEL_NUMBER",
    "CELL_NUMBER",
    "PANEL_NAME",
    "COUNT_GIPRCNT",
    "AVERAGE_GIPRCNT",
    "STDDEV_GIPRCNT",
    "COUNT_PTC",
    "STDDEV_PTC"
], axis=1)
dose_resp = dose_resp[dose_resp["CONCENTRATION_UNIT"] == "M"]
dose_resp.head()

,EXPID,NSC,CONCENTRATION_UNIT,LOG_HI_CONCENTRATION,CONCENTRATION,CELL_NAME,PANEL_CODE,AVERAGE_PTC
0,0001MD02,123127,M,-4.6021,-4.6021,HOP-62,LNS,24.4469
1,0001MD02,123127,M,-4.6021,-4.6021,A549/ATCC,LNS,8.4318
2,0001MD02,123127,M,-4.6021,-4.6021,T-47D,BRE,38.5507
3,0001MD02,123127,M,-4.6021,-4.6021,K-562,LEU,14.3695
4,0001MD02,123127,M,-4.6021,-5.6021,EKVX,LNS,57.3222


In [3]:
def dose_response(x, logIC50, Hillslope):
    return 100 / (1 + 10 ** ((logIC50 - x) * Hillslope))

def remove_outliers(df):
    df['Outlier'] = False
    df_sorted = df.sort_values(by='CONCENTRATION')
    for conc, group in df_sorted.groupby('CONCENTRATION'):
        Q1 = group['AVERAGE_PTC'].quantile(0.25)
        Q3 = group['AVERAGE_PTC'].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        outliers = group[(group['AVERAGE_PTC'] < lower_bound) | (group['AVERAGE_PTC'] > upper_bound)]
        df.loc[outliers.index, 'Outlier'] = True
    return df

def calculate_IC50(df):
    # Group by NSC and CELL_NAME
    grouped = df.groupby(['NSC', 'CELL_NAME'])
    
    IC50_values = []
    for group_key, group_df in tqdm(grouped, total=len(grouped), desc="Calculating IC50"):
        # Remove outliers within each group
        group_df = remove_outliers(group_df)
        
        if len(group_df) == 0:
            continue
        
        concentrations = group_df['CONCENTRATION']
        responses = group_df['AVERAGE_PTC']

        initial_guess = (0, -1)
        try:
            params, covariance = curve_fit(dose_response, concentrations, responses, p0=initial_guess)
        except RuntimeError:
            continue

        fitted_responses = dose_response(concentrations, *params)
        rmse = np.sqrt(mean_squared_error(responses, fitted_responses))
            
        logIC50 = params[0]
        HillSlope = params[1]

        IC50_values.append({
            'NSC': group_key[0],
            'CELL_NAME': group_key[1],
            'logIC50': logIC50,
            'HillSlope': HillSlope,
            "RMSE": rmse
        })

    IC50_df = pd.DataFrame(IC50_values)

    return IC50_df

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
responses_exp = calculate_IC50(dose_resp)

Calculating IC50:   0%|          | 835/3251600 [01:06<71:53:31, 12.56it/s] 


KeyboardInterrupt: 

In [15]:
responses_exp.head()

,EXPID,NSC,CELL_NAME,logIC50,HillSlope,RMSE
0,0001MD02,19893,A498,-0.722969,-0.235588,2.775089
1,0001MD02,19893,A549/ATCC,-4.190503,-0.306085,10.192824
2,0001MD02,19893,ACHN,-3.399132,-0.317039,6.586397
3,0001MD02,19893,BT-549,0.172110,-0.128231,2.126677
4,0001MD02,19893,CAKI-1,-3.257086,-0.241409,6.647007
